In [ ]:
import os
import glob

import numpy as np
import pandas as pd
import xarray as xr

import dask.dataframe as dd
from scipy.spatial import Delaunay
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import cartopy.crs as ccrs
import cartopy.feature as cfeature

#import metpy.calc as mpcalc
#from metpy.units import units

import hvplot.xarray
#import hvplot

from uviz.utils.tools import find_TC_bbox
#from uviz.datashader_tools.utils import datashader_wrapper
from uviz.plotting.utils import basin_bboxes, ssh_wsp, ssh_mslp, sshws_color, geog_features

# import holoviews as hv
# from holoviews import opts

# import geoviews as gv
# import geoviews.feature as gf

# import datashader as ds
# from holoviews.operation.datashader import rasterize as hds_rasterize
# from holoviews.operation.datashader import datashade as hds_datashade

# gv.extension("bokeh","matplotlib")
# #hv.extension("bokeh","matplotlib")

# opts.defaults()

In [ ]:
prototype_dir = r"/gpfs/group/cmz5202/default/cnd5285/synth_events"
storm_1279_dir = os.path.join(prototype_dir, "v0.1/VR28.NATL.EXT.CAM5.4CLM5.0.dtime900_storm_1279")
parent_28km = os.path.join(storm_1279_dir, '28km')
child_3km = os.path.join(storm_1279_dir, '3km')

In [ ]:
h1p_files = glob.glob(os.path.join(parent_28km, '*.h1.*.nc'))
h2p_files = glob.glob(os.path.join(parent_28km, '*.h2.*.nc'))
h3p_files = glob.glob(os.path.join(parent_28km, '*.h3.*.nc'))
h4p_files = glob.glob(os.path.join(parent_28km, '*.h4.*.nc'))

# CAM mesh is the "ext" one, based on original model run. Other options are "ref" or "wat"
p_mesh = r"/gpfs/group/cmz5202/default/cnd5285/maps_and_grids/ne0np4natlanticext.ne30x4.g_scrip.nc"

h1c_files = glob.glob(os.path.join(child_3km, '*.h1.*.nc'))
h2c_files = glob.glob(os.path.join(child_3km, '*.h2.*.nc'))
h3c_files = glob.glob(os.path.join(child_3km, '*.h3.*.nc'))
h4c_files = glob.glob(os.path.join(child_3km, '*.h4.*.nc'))
h5c_files = os.listdir(os.path.join(child_3km, 'h5'))

# Dug in the attributes for the mesh, TODO: make function that finds it automatically
c_mesh = "/gpfs/group/cmz5202/default/cnd5285/MPAS_3km/x20.835586.florida.init.CAM.nc"

In [ ]:
# Native grid files
parallel = True
h1pn_ds = xr.open_mfdataset([f for f in h1p_files if 'remap' not in f], parallel=parallel)
h2pn_ds = xr.open_mfdataset([f for f in h2p_files if 'remap' not in f], parallel=parallel)
h3pn_ds = xr.open_mfdataset([f for f in h3p_files if 'remap' not in f], parallel=parallel)
h4pn_ds = xr.open_mfdataset([f for f in h4p_files if 'remap' not in f], parallel=parallel)
p_mesh_ds = xr.open_dataset(p_mesh)

# Regridded files
h1pr_ds = xr.open_mfdataset([f for f in h1p_files if 'remap' in f], parallel=parallel)
h2pr_ds = xr.open_mfdataset([f for f in h2p_files if 'remap' in f], parallel=parallel)
h3pr_ds = xr.open_mfdataset([f for f in h3p_files if 'remap' in f], parallel=parallel)
h4pr_ds = xr.open_mfdataset([f for f in h4p_files if 'remap' in f], parallel=parallel)

# Native grid files
h1cn_ds = xr.open_mfdataset([f for f in h1c_files if 'remap' not in f], parallel=parallel)
h2cn_ds = xr.open_mfdataset([f for f in h2c_files if 'remap' not in f], parallel=parallel)
h3cn_ds = xr.open_mfdataset([f for f in h3c_files if 'remap' not in f], parallel=parallel)
h4cn_ds = xr.open_mfdataset([f for f in h4c_files if 'remap' not in f], parallel=parallel)
#h5cn_ds = xr.open_mfdataset([f for f in h5c_files])
c_mesh_ds = xr.open_dataset(c_mesh, chunks={'nCells':10000, 'nVertices':100, 'nEdges':100})

# Regridded files
h1cr_ds = xr.open_mfdataset([f for f in h1c_files if 'remap' in f], parallel=parallel)
h2cr_ds = xr.open_mfdataset([f for f in h2c_files if 'remap' in f], parallel=parallel)
h3cr_ds = xr.open_mfdataset([f for f in h3c_files if 'remap' in f], parallel=parallel)
h4cr_ds = xr.open_mfdataset([f for f in h4c_files if 'remap' in f], parallel=parallel)

In [ ]:
def T_to_FLUT(T, unit='K'):
    if unit == 'C':
        T += 273.15
    sigma = 5.6693E-8
    olr = sigma*(T**4)
    
    return olr

In [ ]:
# Normalized cimss scale
bw_colors = [(0, '#BCBCBC'), (1, '#000000')]  
bw_cmp = LinearSegmentedColormap.from_list('FLUT bw', bw_colors, N=435)

levels = np.array([T_to_FLUT(temp, 'C') for temp in [-110, -105, -87.5, -80, -70, -60, -50, -35, -27.5, -22.5]])
fracs = levels-T_to_FLUT(-110, 'C')

rainbow_colors = [(0, '#0febff'), # cyan
                  ((fracs[1]/fracs[-1]), '#7f007f'), # purple
                  ((fracs[2]/fracs[-1]), '#e5e4e5'), # white
                  ((fracs[3]/fracs[-1]), '#000000'), # black
                  ((fracs[4]/fracs[-1]), '#ff0000'), # red
                  ((fracs[5]/fracs[-1]), '#FFFF00'), # yellow
                  ((fracs[6]/fracs[-1]), '#00FF00'), # green
                  ((fracs[7]/fracs[-1]), '#000073'), # navy
                  (1, '#00ffff')] # cyan


rainbow_cmp = LinearSegmentedColormap.from_list('FLUT colors', rainbow_colors, N=184)

bws = plt.get_cmap(bw_cmp)
bws_colors = bws(np.linspace(0, 1, 435))
rainbow = plt.get_cmap(rainbow_cmp)
r_colors = rainbow(np.linspace(0, 1, 184))

all_colors = np.vstack((r_colors, bws_colors))
flut_cimss = LinearSegmentedColormap.from_list('FLUT_CIMSS', all_colors)
mpl.colormaps.register(flut_cimss, name='FLUT_CIMSS', force=True)

# hvplot

In [ ]:
def unzipMesh(x,tris,t):
    return tris[(np.abs((x[tris[:,0]])-(x[tris[:,1]])) < t) & (np.abs((x[tris[:,0]])-(x[tris[:,2]])) < t)]

def triArea(x,y,tris):
    return ((x[tris[:,1]]-x[tris[:,0]]) * (y[tris[:,2]]-y[tris[:,0]])) - ((x[tris[:,2]]-x[tris[:,0]]) * (y[tris[:,1]]-y[tris[:,0]]))

def orderCCW(x,y,tris):
    tris[triArea(x,y,tris)<0.0,:] = tris[triArea(x,y,tris)<0.0,::-1]
    return(tris)

def createHVTriMesh(x,y,triangle_indices, var, var_name, n_workers=1):
    # Declare verts array
    # This is essentally an XYZ matrix (at the location of x and y, z=?) 3 X len(verts) array
    verts = np.column_stack([x, y, var])

    # Convert to pandas
    verts_df  = pd.DataFrame(verts,  columns=['Longitude', 'Latitude', var_name])
    tris_df   = pd.DataFrame(triangle_indices, columns=['v0', 'v1', 'v2'])

    # Convert to dask
    verts_ddf = dd.from_pandas(verts_df, npartitions=n_workers)
    tris_ddf = dd.from_pandas(tris_df, npartitions=n_workers)

    # Declare HoloViews element
    tri_nodes = hv.Nodes(verts_ddf, ['Longitude', 'Latitude', 'index'], [var_name])
    trimesh = hv.TriMesh((tris_ddf, tri_nodes))
    return(trimesh)

lonCell = np.mod(np.rad2deg(c_mesh_ds['lonCell'].values) - 180.0, 360.0) - 180.0
latCell = np.rad2deg(c_mesh_ds['latCell'].values)
lonVertex = np.mod(np.rad2deg(c_mesh_ds['lonVertex'].values) - 180.0, 360.0) - 180.0
latVertex = np.rad2deg(c_mesh_ds['latVertex'].values)
tris = c_mesh_ds.cellsOnVertex.values - 1
tris_ccw = orderCCW(lonCell,latCell,tris)
tris_ccw_flat = unzipMesh(lonCell,tris,90.0)

sel_time = 23
primalVar = h3cn_ds['FLUT'].isel(time=sel_time).values
lon_range, lat_range = find_TC_bbox(h3cn_ds, 'florida', sel_time, center_dist=1000)

project_coords = True
if project_coords == True:
    proj = ccrs.Robinson(central_longitude=0.0)
    xPCS, yPCS, _ = proj.transform_points(ccrs.PlateCarree(), lonCell, latCell).T
    trimesh = createHVTriMesh(xPCS,yPCS,tris_ccw_flat, primalVar, 'FLUT', n_workers=1)
    x_range, y_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
    lon_range = tuple(x_range)
    lat_range = tuple(y_range)

else:
    proj = ccrs.PlateCarree()
    lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))
    trimesh = createHVTriMesh(lonCell,latCell,tris_ccw, primalVar, 'FLUT', n_workers=1)

rasterized = hds_rasterize(trimesh, aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)

mpas_native_olr = rasterized.opts(tools=['hover'], colorbar=True, cmap=flut_cimss, title='MPAS (Native) Upwelling Longwave Flux', 
                           clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), width=1000, height=900, clabel='Upwelling Longwave Flux [W/m^2]',
                           colorbar_position='bottom', fontsize=dict(title='30pt', clabel='28pt', cticks='26pt')) * gf.coastline(projection=proj).options(scale='10m')

In [ ]:
def triangulate(vertices, x="Longitude", y="Latitude"):
    """
    Generate a triangular mesh for the given x,y,z vertices, using Delaunay triangulation.
    For large n, typically results in about double the number of triangles as vertices.
    """
    triang = Delaunay(vertices[[x,y]].values)
    print('Given', len(vertices), "vertices, created", len(triang.simplices), 'triangles.')
    
    tris_df = pd.DataFrame(triang.simplices, columns=['v0', 'v1', 'v2'])
    
    return tris_df

def createHVTriMesh(x, y, var, var_name, n_workers=1):
    # Declare verts array
    # This is essentally an XYZ matrix (at the location of x and y, z=?) 3 X len(verts) array
    verts = np.column_stack([x, y, var])

    # Convert to pandas
    verts_df  = pd.DataFrame(verts,  columns=['Longitude', 'Latitude', var_name])
    
    # Creates Delaunay triangular mesh
    tris_df = triangulate(verts_df)

    # Convert to dask
    verts_ddf = dd.from_pandas(verts_df, npartitions=n_workers)
    tris_ddf = dd.from_pandas(tris_df, npartitions=n_workers)

    # Declare HoloViews element
    tri_nodes = hv.Nodes(verts_ddf, ['Longitude', 'Latitude', 'index'], [var_name])
    trimesh = hv.TriMesh((tris_ddf, tri_nodes))
    return(trimesh)

# Grid centerpoints
lonCell = p_mesh_ds.grid_center_lon.values
latCell = p_mesh_ds.grid_center_lat.values
lonCell = ((lonCell - 180.0) % 360.0) - 180.0

sel_time = 23
primalVar = h3pn_ds['FLUT'].isel(time=sel_time).values
lon_range, lat_range = find_TC_bbox(h3pn_ds, 'florida', sel_time, center_dist=1000)

project_coords = True

if project_coords == True:
    proj = ccrs.Robinson(central_longitude=0.0)
    # Creates Delaunay triangular mesh
    xPCS, yPCS, _ = proj.transform_points(ccrs.PlateCarree(), lonCell, latCell).T
    trimesh = createHVTriMesh(xPCS, yPCS, primalVar, 'FLUT')
    x_range, y_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
    lon_range = tuple(x_range)
    lat_range = tuple(y_range)
    
else:
    proj = ccrs.PlateCarree()
    lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))
    # Creates Delaunay triangular mesh
    trimesh = createHVTriMesh(lonCell, latCell, primalVar, 'FLUT')


rasterized = hds_rasterize(trimesh, aggregator='mean', precompute=True, x_range=lon_range, y_range=lat_range, vdim_prefix='')

cam_native_olr = rasterized.opts(tools=['hover'], colorbar=False, cmap=flut_cimss, title='CAM5 (Native Grid) Upwelling Longwave Flux', 
                        clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), width=1000, height=800, 
                        fontsize=dict(title='30pt')) * gf.coastline(projection=proj).options(scale='10m')

In [ ]:
x_range

In [ ]:
lon_range, lat_range = find_TC_bbox(h3cn_ds, 'florida', 23, center_dist=1000)
lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))
proj = ccrs.Robinson(central_longitude=0.0)

w = 1000
h = 600

mpas_regridded_olr = h3cr_ds.FLUT.isel(time=23).hvplot('lon', 'lat', projection=proj, project=True, rasterize=True, aggregator='mean', colorbar=False,
                                           clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), cmap='FLUT_CIMSS', title='MPAS (Regridded) Upwelling Longwave Flux',
                                           dynamic=False, xlim = lon_range, ylim = lat_range, coastline='10m', fontsize=dict(title='30pt'),
                                           frame_width=w, frame_height=h)
mpas_regridded_olr

In [ ]:
# Not Robinson!
# lon_ and lat_ range need to be in degrees and [-180, 180]

lon_range, lat_range = find_TC_bbox(h3cn_ds, 'florida', 23, center_dist=1000)
#lon_range, lat_range = basin_bboxes('florida')
#lon_range = tuple((-88, -65))
#lat_range = tuple((16.13, 34.32))
lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))
proj = ccrs.Robinson()

w = 1000
h = 600

cam_regridded_olr = h3pr_ds.FLUT.isel(time=23).hvplot.quadmesh('lon', 'lat', projection=proj, project=True, rasterize=True, aggregator='mean', colorbar=False,
                                           clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), cmap='FLUT_CIMSS', title='CAM (Regridded) Upwelling Longwave Flux',
                                           dynamic=False, xlim = lon_range, ylim = lat_range, coastline='10m', fontsize=dict(title='30pt'),
                                           frame_width=w, frame_height=h)
cam_regridded_olr

In [ ]:
hvplot.help('quadmesh')

In [ ]:
lon_range

In [ ]:
right_olr = hv.Layout([cam_regridded_olr, mpas_regridded_olr]).cols(1)

In [ ]:
final_olr = hv.Layout([cam_native_olr, cam_regridded_olr, mpas_native_olr, mpas_regridded_olr]).cols(2)

In [ ]:
hv.save(final_olr, '../figs/olr_final.png', fmt='png')

In [ ]:
hv.save(right_olr, '../figs/olr_regridded_right.png', fmt='png')

In [ ]:
# FLUT CIMSS is my registered mpl colormap
project_coords=True

if project_coords == True:
    proj = ccrs.Robinson(central_longitude=180)
    h3cr_ds.FLUT.isel(time=slice(20, 30)).hvplot.quadmesh('lon', 'lat', projection=proj, project=True, rasterize=True, clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')),
                             cmap='FLUT_CIMSS', dynamic=False, coastline='10m', frame_width=1200, frame_height=800)
else:
    h3cr_ds.FLUT.isel(time=slice(20, 30)).hvplot.quadmesh('lon', 'lat', geo=True, rasterize=True, clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')),
                             cmap='FLUT_CIMSS', dynamic=False, coastline='10m', frame_width=1200, frame_height=800)

In [ ]:
proj = ccrs.Robinson(central_longitude=180.0)
lon_range, lat_range = find_TC_bbox(h3cn_ds, 'florida', 23, center_dist=1000)
xPCS, yPCS, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
tuple(xPCS)

In [ ]:
lon_range, lat_range = find_TC_bbox(h3cn_ds, 'florida', 23, center_dist=1000)
lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))

proj = ccrs.Robinson()
# lon_range, lat_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
# lon_range = tuple(lon_range)
# lat_range = tuple(lat_range)

w = 800
h = 600

h3cr_ds.FLUT.isel(time=23).hvplot.quadmesh('lon', 'lat', projection=proj, project=True, rasterize=True, aggregator='mean',
                                           clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), cmap='FLUT_CIMSS', title='Upwelling Longwave Flux MPAS (Regridded)',
                                           dynamic=False, xlim = lon_range, ylim = lat_range, coastline='10m', fontsize=dict(title='16pt', cticks='10pt'),
                                           min_width=w, max_width=w, min_height=h, max_height=h, clabel='Upwelling Longwave Flux [W/m^2]')

In [ ]:
lon_range, lat_range = find_TC_bbox(h3cn_ds, 'florida', 23, center_dist=1000)
lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))

proj = ccrs.Robinson()
# lon_range, lat_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
# lon_range = tuple(lon_range)
# lat_range = tuple(lat_range)

w = 800
h = 600

h3cr_ds.FLUT.isel(time=23).hvplot.quadmesh('lon', 'lat', projection=proj, project=True, rasterize=True, aggregator='mean',
                                           clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), cmap='FLUT_CIMSS', title='Upwelling Longwave Flux MPAS (Regridded)',
                                           dynamic=False, xlim = lon_range, ylim = lat_range, coastline='10m', fontsize=dict(title='16pt', cticks='10pt'),
                                           min_width=w, max_width=w, min_height=h, max_height=h, clabel='Upwelling Longwave Flux [W/m^2]')

proj = ccrs.Robinson(central_longitude=0.0)
h3pr_ds.FLUT.isel(time=23).hvplot.quadmesh('lon', 'lat', projection=proj, project=True, rasterize=True, aggregator='mean',
                                           clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), cmap='FLUT_CIMSS', title='Upwelling Longwave Flux CAM5 (Regridded)',
                                           dynamic=False, xlim = lon_range, ylim = lat_range, coastline='10m', fontsize=dict(title='16pt', cticks='10pt'),
                                           min_width=w, max_width=w, min_height=h, max_height=h, clabel='Upwelling Longwave Flux [W/m^2]')

In [ ]:
#lon_range, lat_range = find_TC_bbox(h3cn_ds, 'florida', 23, center_dist=1000)
# slice(20, 30)
h3pr_ds.FLUT.isel(time=23).hvplot.quadmesh('lon', 'lat', geo=True, rasterize=True, min_width=800, max_width=800,
                                           min_height=600, max_height=600, cmap='gist_yarg', fontsize=dict(title='16pt', cticks='11pt'))

In [ ]:
lon_range, lat_range = find_TC_bbox(h3cn_ds, 'florida', 23, center_dist=1000)
lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))

proj = ccrs.Robinson()
# lon_range, lat_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
# lon_range = tuple(lon_range)
# lat_range = tuple(lat_range)

w = 800
h = 600

h3cr_ds.FLUT.isel(time=23).hvplot.quadmesh('lon', 'lat', projection=proj, project=True, rasterize=True, aggregator='mean',
                                           clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), cmap='FLUT_CIMSS', title='Upwelling Longwave Flux MPAS (Regridded)',
                                           dynamic=False, xlim = lon_range, ylim = lat_range, coastline='10m', fontsize=dict(title='16pt', cticks='10pt'),
                                           min_width=w, max_width=w, min_height=h, max_height=h, clabel='Upwelling Longwave Flux [W/m^2]')
#lon_range, lat_range = find_TC_bbox(h3cn_ds, 'florida', 23, center_dist=1000)
# slice(20, 30)
h3pr_ds.FLUT.isel(time=23).hvplot.quadmesh('lon', 'lat', geo=True, rasterize=True, min_width=800, max_width=800,
                                           min_height=600, max_height=600, cmap='gist_yarg', fontsize=dict(title='16pt', cticks='11pt'))

In [ ]:
lon_range

In [ ]:
lat_range

In [ ]:
h3pr_ds.FLUT.isel(time=23).hvplot.quadmesh('lon', 'lat', geo=True, rasterize=True, clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')),
                             cmap = 'FLUT_CIMSS', dynamic=False, coastline='10m', width=800, height=600)

In [ ]:
lons, lats = basin_bboxes('florida')

fig, axs = plt.subplots(2, 1, dpi=200, figsize=(15, 12), subplot_kw=dict(projection=ccrs.PlateCarree()), layout='constrained')

norm = mpl.colors.Normalize(vmin=T_to_FLUT(-110, 'C'), vmax=T_to_FLUT(55, 'C'))
h3pr_ds['FLUT'].isel(time=23).plot(ax=axs[0], norm=norm, cmap='FLUT_CIMSS', add_colorbar=False, transform=ccrs.PlateCarree())
h3cr_ds['FLUT'].isel(time=23).plot(ax=axs[1], norm=norm, cmap='FLUT_CIMSS', add_colorbar=False, transform=ccrs.PlateCarree())
axs[0].set_title('CAM5 (Regridded) Upwelling Longwave Flux', fontweight='bold', fontsize=16)
axs[1].set_title('MPAS (Regridded) Upwelling Longwave Flux', fontweight='bold', fontsize=16)

for ax in axs.ravel():
    ax.set_extent([lons[0], lons[1], lats[0], lats[1]])
    ax.coastlines(resolution='10m', linewidth=1, edgecolor='#323232')
    ax.add_feature(cfeature.LAKES.with_scale('10m'), linewidth=1, facecolor='none', edgecolor='#323232', zorder=2)
    
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='FLUT_CIMSS'), orientation='horizontal', ax=axs, shrink=0.375)
cbar.ax.tick_params(labelsize=14)
cbar.set_label(label='Upwelling Longwave Flux [W/$m^2$]', size=16, fontstyle='italic')
    
# cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='FLUT_CIMSS'), orientation='horizontal', shrink = 
#              ax=axs)

plt.show()

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

In [ ]:
p_mesh_ds

In [ ]:
center_dist = 1000 # km
lon_range, lat_range = find_TC_bbox(h3pn_ds, 'florida', 23, center_dist=center_dist)
lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))

x = p_mesh_ds.grid_center_lon.values
x = ((np.subtract(x, 180) % 360) - 180.0)
y = p_mesh_ds.grid_center_lat.values
var = h3pn_ds.FLUT.isel(time=23).values

points = np.column_stack([x, y, var])
points = points[np.where((points[:, 0] >= lon_range[0]) & (points[:, 0] <= lon_range[1]) & 
                         (points[:, 1] >= lat_range[0]) & (points[:, 1] <= lat_range[1]))]

grid_x, grid_y = np.meshgrid(points[:, 0], points[:, 1], indexing='xy', sparse=False)

In [ ]:
tvar = np.dstack([points[:, 2]]*len(points[:, 2]))
tvar.shape

In [ ]:
tvar.shape

In [ ]:
grid_x.shape

In [ ]:
points[:, 2].shape

In [ ]:
hv.QuadMesh((grid_x, grid_y, tvar))

In [ ]:
# Based off of https://water.weather.gov/precip/ 30-day observed precip scale
nws_precip_colors = [
    '#ffffff',  # 0 - 0.01 inches
    "#4bd2f7",  # 0.01 - 0.10 inches  light blue
    "#699fd0",  # 0.10 - 0.25 inches  mid blue
    "#3c4bac",  # 0.25 - 0.50 inches  dark blue
    "#3cf74b",  # 0.50 - 1.00 inches  light green
    "#3cb447",  # 1.00 - 1.50 inches  mid green
    "#3c8743",  # 1.50 - 2.00 inches  dark green
    "#1f4723",  # 2.00 - 3.00 inches  darkest green
    "#f7f73c",  # 3.00 - 4.00 inches  yellow
    "#fbde88",  # 4.00 - 5.00 inches  weird tan
    "#f7ac3c",  # 5.00 - 6.00 inches  orange
    "#c47908",  # 6.00 - 8.00 inches  dark orange
    "#f73c3c",  # 8.00 - 10.00 inch   red
    "#bf3c3c",  # 10.00 - 15.00 inch  mid red
    "#6e2b2b",  # 15.00 - 20.00 inch  dark red 
    "#f73cf7",  # 20.00 - 25.00 inch  bright pink
    "#9974e4",  # 25.00 - 30.00 inch  purple
    "#e1e1e1",  # 30.00 - 40.00 inch  gray
    ]
nws_cmap = mpl.colors.ListedColormap(nws_precip_colors, 'nws_precip')
levels = [0.0, 0.01, 0.10, 0.25, 0.50, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 10.0, 15.0, 20.0, 25.0, 30.0, 40.0]
norm = mpl.colors.BoundaryNorm(boundaries=levels, ncolors=len(levels))
mpl.colormaps.register(nws_cmap, force=True)
nws_cmap

In [ ]:
h4cr_ds['tot_prect'] = h4cr_ds['PRECT'].metpy.convert_units('inch/hour').sum(dim='time') * units('hour')
h4cr_ds['max_hrly_prect'] = h4cr_ds['PRECT'].metpy.convert_units('inch/hour').max(dim='time') * units('hour')

h4pr_ds['tot_prect'] = h4pr_ds['PRECT'].metpy.convert_units('inch/hour').sum(dim='time') * units('hour')
h4pr_ds['max_hrly_prect'] = h4pr_ds['PRECT'].metpy.convert_units('inch/hour').max(dim='time') * units('hour')

In [ ]:
h4cr_ds

In [ ]:
lon_range, lat_range = basin_bboxes('florida')
lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))

w = 800
h = 600

proj = ccrs.Robinson(central_longitude=0.0)
h4pr_ds.tot_prect.hvplot.quadmesh('lon', 'lat', projection=proj, project=True, rasterize=True, aggregator='mean',
                                           clim=(levels[0], levels[-1]), cmap='nws_precip', title='CAM5 (Regridded) Total Accumulated Precipitation',
                                           dynamic=False, xlim = lon_range, ylim = lat_range, coastline='10m', fontsize=dict(title='16pt', cticks='10pt'),
                                           min_width=w, max_width=w, min_height=h, max_height=h, clabel='Accumulated Precipitation [inches]', alpha=0.75)


# final = rasterized.opts(tools=['hover'], colorbar=True, cmap=nws_cmap, title='MPAS (Native) Maximum Hourly Accumulated Precipitation', 
#                         clim=(levels[0], levels[-1]), width=1000, clabel='Accumulated Precipitation [inches]', color_levels=levels,
#                         clipping_colors={'NaN':'#FFFFFF'}, fontsize=dict(title='16pt', cticks='10pt'), alpha=0.75) *\
# gf.coastline(projection=proj, line_width=1, alpha=1).options(scale='10m') *\
# gf.lakes(projection=proj, line_width=0.5, fill_color='none', line_color='#323232', alpha=1).options(scale='10m')

# Scipy interpolation

In [ ]:
from scipy.interpolate import griddata, Rbf

In [ ]:
p_mesh_ds

In [ ]:
h3cn_ds

In [ ]:
center_dist = 500 # km
lon_range, lat_range = find_TC_bbox(h3pn_ds, 'florida', 23, center_dist=center_dist)
lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))

x = p_mesh_ds.grid_center_lon.values
x = ((np.subtract(x, 180) % 360) - 180.0)
y = p_mesh_ds.grid_center_lat.values
var = h3pn_ds.FLUT.isel(time=23).values

points = np.column_stack([x, y, var])
points = points[np.where((points[:, 0] >= lon_range[0]) & (points[:, 0] <= lon_range[1]) & 
                         (points[:, 1] >= lat_range[0]) & (points[:, 1] <= lat_range[1]))]

In [ ]:
import time
start = time.time()
grid_x, grid_y = np.meshgrid(points[:, 0], points[:, 1], indexing='xy', sparse=True)
end = time.time()

print(f'Time elapsed with sparse=True: {end-start} seconds.')

In [ ]:
start = time.time()
grid_x, grid_y = np.meshgrid(points[:, 0], points[:, 1], indexing='xy', sparse=False)
end = time.time()

print(f'Time elapsed with sparse=False: {end-start} seconds.')

In [ ]:
nearest_grid = griddata(points[:, :2], points[:, 2], (grid_x, grid_y), method='nearest')
linear_grid = griddata(points[:, :2], points[:, 2], (grid_x, grid_y), method='linear')
cubic_grid = griddata(points[:, :2], points[:, 2], (grid_x, grid_y), method='cubic') 

# try:
#     rbf = Rbf(points[:, 0], points[:, 1], points[:, 2])
#     rbf_grid = rbf(grid_x,grid_y)
# except:
#     rbf_grid = None

In [ ]:
proj = ccrs.PlateCarree()
fig, axs = plt.subplots(1, 3, figsize=(12, 6), dpi=200, subplot_kw=dict(projection=proj))

# axs[0][0].imshow(rbf_grid, origin='lower', 
#            extent=(points[:, 0].min(), points[:, 0].max(), points[:, 1].min(), points[:, 1].max()),
#            aspect=(points[:, 0].max() - points[:, 0].min()) / (points[:, 1].max() - points[:, 1].min()),
#            vmin=T_to_FLUT(-110, 'C'), vmax=T_to_FLUT(55, 'C'), cmap='FLUT_CIMSS', transform=proj)
# axs[0][0].set_title('RBF Interp')

axs[0].imshow(nearest_grid, origin='lower', 
           extent=(points[:, 0].min(), points[:, 0].max(), points[:, 1].min(), points[:, 1].max()),
           aspect=(points[:, 0].max() - points[:, 0].min()) / (points[:, 1].max() - points[:, 1].min()),
           vmin=T_to_FLUT(-110, 'C'), vmax=T_to_FLUT(55, 'C'), cmap='FLUT_CIMSS', transform=proj)
axs[0].set_title('Nearest Neighbor Interp')

axs[1].imshow(linear_grid, origin='lower', 
           extent=(points[:, 0].min(), points[:, 0].max(), points[:, 1].min(), points[:, 1].max()),
           aspect=(points[:, 0].max() - points[:, 0].min()) / (points[:, 1].max() - points[:, 1].min()),
           vmin=T_to_FLUT(-110, 'C'), vmax=T_to_FLUT(55, 'C'), cmap='FLUT_CIMSS', transform=proj)
axs[1].set_title('Linear Interp')

axs[2].imshow(cubic_grid, origin='lower', 
           extent=(points[:, 0].min(), points[:, 0].max(), points[:, 1].min(), points[:, 1].max()),
           aspect=(points[:, 0].max() - points[:, 0].min()) / (points[:, 1].max() - points[:, 1].min()),
           vmin=T_to_FLUT(-110, 'C'), vmax=T_to_FLUT(55, 'C'), cmap='FLUT_CIMSS', transform=proj)
axs[2].set_title('Cubic Interp')

for ax in axs.ravel():
    ax.coastlines()

plt.show()

In [ ]:
rbf = Rbf(points[:, 0], points[:, 1], points[:, 2])
rbf_grid = rbf(grid_x,grid_y)

#come back to this

In [ ]:
points.shape

In [ ]:
from matplotlib.mlab import griddata

# Plotly

In [ ]:
h1cn_ds

In [ ]:
hvplot.extension('plotly')

In [ ]:
df = h3cn_ds['FLUT'].to_dataframe()
df

# PyNGL

# Paul's NCL Code